<a href="https://colab.research.google.com/github/SamGrobelny/ASTRON1221/blob/main/transit_stuff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii
from astropy.timeseries import TimeSeries
from scipy.optimize import curve_fit

# Read the transit data
data = ascii.read('hd209458b_transit.txt')

# Extract time and flux, and normalize flux
time = data['time']
flux = data['flux']
flux_err = data['flux_err']
if 'flux_err' not in data.colnames:
    flux_err = np.full_like(flux, np.std(flux))

# Plot the raw data
plt.figure(figsize=(10, 5))
plt.errorbar(time, flux, yerr=flux_err, fmt='k.', ecolor='gray', alpha=0.5)
plt.xlabel('Time [BJD]')
plt.ylabel('Normalized Flux')
plt.title('HD 209458 b Transit Light Curve')
plt.show()

# Define the box transit model
def box_model(t, t0, duration, depth, baseline):
    """Simple box-shaped transit model."""
    model = np.full_like(t, baseline)
    half_duration = duration / 2.0
    in_transit = np.abs(t - t0) < half_duration
    model[in_transit] -= depth
    return model

# Initial guess parameters
initial_t0 = np.median(time)
initial_duration = 0.1  # in same units as time
initial_depth = 0.015  # Approximate depth of transit
initial_baseline = 1.0

initial_params = [initial_t0, initial_duration, initial_depth, initial_baseline]

# Fit the model to the data
popt, pcov = curve_fit(
    box_model, time, flux, p0=initial_params, sigma=flux_err, absolute_sigma=True
)

# Extract fitted parameters
fitted_t0, fitted_duration, fitted_depth, fitted_baseline = popt
fitted_params = popt
param_errors = np.sqrt(np.diag(pcov))

# Generate model flux using fitted parameters
model_flux = box_model(time, *fitted_params)

# Plot data with fitted model
plt.figure(figsize=(10, 5))
plt.errorbar(time, flux, yerr=flux_err, fmt='k.', ecolor='gray', alpha=0.5, label='Data')
plt.plot(time, model_flux, 'r-', label='Fitted Box Model')
plt.xlabel('Time [BJD]')
plt.ylabel('Normalized Flux')
plt.title('HD 209458 b Transit Fit')
plt.legend()
plt.show()

# Calculate reduced Chi-squared
def reduced_chi_squared(observed, expected, errors, num_params):
    chi2 = np.sum(((observed - expected) / errors) ** 2)
    dof = len(observed) - num_params
    rchi2 = chi2 / dof
    return rchi2

num_params = len(fitted_params)
rchi2 = reduced_chi_squared(flux, model_flux, flux_err, num_params)
print(f"Reduced Chi-squared: {rchi2:.2f}")

# Try different model parameters and report reduced Chi-squared values
durations = [fitted_duration * 0.8, fitted_duration, fitted_duration * 1.2]
depths = [fitted_depth * 0.8, fitted_depth, fitted_depth * 1.2]
t0s = [fitted_t0 - 0.01, fitted_t0, fitted_t0 + 0.01]

print("\nTesting different parameters:")
for duration in durations:
    for depth in depths:
        for t0 in t0s:
            test_params = [t0, duration, depth, fitted_baseline]
            test_model_flux = box_model(time, *test_params)
            test_rchi2 = reduced_chi_squared(flux, test_model_flux, flux_err, num_params)
            print(
                f"t0: {t0:.5f}, Duration: {duration:.5f}, Depth: {depth:.5f}, "
                f"Reduced Chi-squared: {test_rchi2:.2f}"
            )


FileNotFoundError: [Errno 2] No such file or directory: 'hd209458b_transit.txt'